In [42]:
#Import dependencies and setup
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from IPython.display import HTML

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\aleja\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


## Step 1 - Scraping

### NASA Mars News

In [3]:
# Connecting to the NASA
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
#Scrape the Mars News Site 
html = browser.html
news_soup = BeautifulSoup(html, "html.parser")
slide_element = news_soup.select_one("ul.item_list li.slide")

In [5]:
# Scrape the Latest News Title
news_title = slide_element.find("div", class_="content_title").get_text()
print(news_title)

Clays, Not Water, Are Likely Source of Mars 'Lakes'


In [6]:
# Scrape the Latest News Paragraph
news_paragraph = slide_element.find("div", class_="article_teaser_body").get_text()
print(news_paragraph)

Three studies published in the past month have cast doubt on the premise of subsurface lakes below the Martian south pole.


### JPL Mars Space Images

In [20]:
# Connecting to the NASA
url = "https://www.jpl.nasa.gov/images?query=Mars"
browser.visit(url)

In [21]:
#Find the image url for the current Featured Mars Image and assign the url string to a variable
full_image_button = browser.find_by_id("SearchListingPageResults")
full_image_button.click()

In [22]:
browser.is_element_present_by_text("Download JPG", wait_time=1)
more_info_element = browser.find_link_by_partial_text("Download JPG")
more_info_element.click()

In [23]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = BeautifulSoup(html, "html.parser")

In [26]:
#Save a complete url string for this image
for link in image_soup.find_all('img'):
    print(link.get('src'))

https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA15964.jpg


In [27]:
#save a complete url string for this image
img_url = image_soup.select_one("img").get("src")
img_url

'https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA15964.jpg'

### Mars Facts




In [41]:
mars_facts = pd.read_html("https://galaxyfacts-mars.com/")[0]
print(mars_facts)
mars_facts.reset_index(inplace=True)
mars_facts.columns=["ID", "Properties", "Mars", "Earth"]
mars_facts

                         0                1                2
0  Mars - Earth Comparison             Mars            Earth
1                Diameter:         6,779 km        12,742 km
2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
3                   Moons:                2                1
4       Distance from Sun:   227,943,824 km   149,598,262 km
5          Length of Year:   687 Earth days      365.24 days
6             Temperature:     -87 to -5 °C      -88 to 58°C


,ID,Properties,Mars,Earth
0,0,Mars - Earth Comparison,Mars,Earth
1,1,Diameter:,"6,779 km","12,742 km"
2,2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,3,Moons:,2,1
4,4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,5,Length of Year:,687 Earth days,365.24 days
6,6,Temperature:,-87 to -5 °C,-88 to 58°C


In [43]:
mars_facts.to_html("mars_facts.html")